In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_google_genai

In [ ]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
import os


In [ ]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

# Initialize Gemini-2.5 Pro LLM
gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,
    top_p=0.95,
)

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	    memory=True,
  llm=gemini_llm,
	verbose=True
)

In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
	 memory=True,
  llm=gemini_llm,
)

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,


)

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought:
The user, Andrew Ng from DeepLearningAI, is asking for help with setting up a Crew and specifically wants to know how to add memory to it.

My goal is to provide the best possible support. This means I need to give a complete, accurate, and friendly answer. I should not make any assumptions and use the a tool I have at my disposal.

The only tool I have is `Read website content`, which can read the content of `https://docs.crewai.com/how-to/Creating-

In [ ]:
from IPython.display import Markdown
Markdown(result)

Hello Andrew,

It's a pleasure to connect with you! My name is [Your Name], and I'm a Senior Support Representative here at crewAI. I'm absolutely thrilled to help you and the DeepLearningAI team with setting up your first Crew. Providing you with the best support is my top priority.

You've asked for guidance on setting up a Crew, kicking it off, and specifically how to add memory. That's a fantastic question, as memory is a powerful feature for enhancing your agents' collaboration. Let's dive into the details.

Based on our official documentation, here is a complete guide to get you started.

### Part 1: Understanding and Setting Up Your Crew

Think of a Crew like a highly efficient team or a small company. It has a manager (the `Crew`), specialized employees (the `Agents`), and a list of things to do (the `Tasks`) that follow a specific workflow (the `Process`).

Here’s a breakdown of the core components as described in our documentation:

*   **AI Agents:** These are your specialized team members. Each agent is designed with a specific role, a backstory or context, a goal, and can be equipped with a set of tools (like search, file reading, etc.). For example, you could have a `ResearcherAgent` and a `WriterAgent`.
    *   **Key Features:** They have specific roles, use designated tools, can delegate tasks, and make autonomous decisions.

*   **Tasks:** These are the individual assignments that you give to your agents. Each task has a clear objective and description. You assign a specific agent to perform each task. The output of one task can be used as input for the next, creating a seamless workflow.
    *   **Key Features:** They have clear objectives, use specific tools, feed into a larger process, and produce actionable results.

*   **The Crew:** This is the top-level orchestrator that brings everything together. You define your agents and tasks and then add them to the Crew. The Crew is responsible for managing the agents and overseeing the workflow to ensure they collaborate effectively to achieve the final goal.

*   **The Process:** This is the workflow management system that defines how your agents collaborate. You can set up processes to be **sequential** (Tasks are executed one after another) or **parallel** (Tasks can be executed simultaneously if they don't depend on each other). This ensures efficient execution and interaction between agents.

### Part 2: Kicking Off Your Crew

Once you have defined your `Agents`, `Tasks`, and assembled them into a `Crew` with a defined `Process`, you are ready to "kick it off"! This is the action that starts the execution. The Crew will manage the agents as they begin working on their assigned tasks, collaborating and sharing information as needed until the final objective is met.

### Part 3: Enabling Memory in Your Crew

This is a crucial step for building more sophisticated and context-aware crews, and I'm glad you asked about it. To enhance collaboration between agents, you can enable memory for your crew. This provides the agents with a shared short-term memory, allowing them to access the context and history from previously executed tasks within the same run. This leads to more coherent and context-aware performance.

You can activate this feature by setting the `memory` parameter to `True` when you instantiate your `Crew`.

Here is the specific code implementation:

```python
# Import the necessary classes from crewAI
from crewai import Crew, Agent, Task

# NOTE: Make sure you have your agents and tasks defined before this step.
# For example:
# planner = Agent(...)
# writer = Agent(...)
# plan_task = Task(...)
# write_task = Task(...)

# Instantiate your crew with memory enabled by setting `memory=True`
my_crew = Crew(
  agents=[planner, writer],
  tasks=[plan_task, write_task],
  memory=True,
  verbose=2 # Set to 1 or 2 for logging; 0 for no logs
)
```

By adding `memory=True`, the `writer` agent in this example will have access to the results and work done by the `planner` agent, ensuring a seamless and informed workflow. It's a simple yet powerful way to improve the crew's overall output.

I hope this detailed explanation is helpful for you and the team at DeepLearningAI! We are incredibly excited to see what you build with crewAI. If any other questions come up as you're getting started, please don't hesitate to reach out. We're here to help you succeed.

Best regards,

Your Friendly Senior Support Representative at crewAI

***

*Reference: The information in this guide was sourced from the official crewAI documentation, including the "How To" and "Core Concepts" sections.*